# YouTube 영상 자동 수집 노트북 v3 (에러 처리 개선)

이 노트북은 YouTube Data API v3를 사용하여:
1. **자동으로 영상을 검색**하고
2. 영상 정보와 댓글을 수집합니다

## ✨ v2 주요 개선사항
- **카테고리별 region 설정 지원**: 각 카테고리마다 다른 지역 선택 가능
- **3단계 우선순위**: 카테고리별 > Trending별 > 전역 region
- **다국어 혼합 데이터셋**: 한 번에 여러 나라 데이터 수집

## 수집 전략
- 방안 1: 카테고리별 인기 영상 검색 (지역별 설정 가능)
- 방안 2: 인기 급상승 영상 (트렌딩)
- 방안 3: 특정 채널의 영상 수집

## PI 할당량 예상
- 현재 사용량 보는 곳    https://console.cloud.google.com/apis/api/youtube.googleapis.com/quotas
- ※ 아래 계산기 활용해서 확인 후 실행

## 설정 가능한 옵션
- `include_no_comments`: 댓글 비활성화 영상 포함 여부 (기본값: True)
- `min_comment_count`: 최소 댓글 수 (기본값: 0)
- `silent_mode`: 경고 메시지 숨김 (기본값: False)

## ✨ v3.1 주요 수정사항 (중요!)
- **카테고리 검색 완전 수정**: `search.list` → `videos.list(chart='mostPopular')` 변경
- **API 비용 99% 절감**: 카테고리별 100 units → 1 unit
- **안정성 대폭 향상**: `videoCategoryId` 정상 작동
- **에러 제거**: `regionCode` 충돌 문제 해결

## 📋 데이터 수집 및 예측 카테고리 정리 (그냥 제가 생각해본 분류입니다)

---

### 🎯 수집할 15개 카테고리 (원본)

| ID | Category Name | 설명 |
|----|----------------|------|
| 1  | Film & Animation | 영화, 애니메이션, 영화 리뷰 |
| 2  | Autos & Vehicles | 자동차, 오토바이, 차량 리뷰 |
| 10 | Music | 음악, 뮤직비디오, 커버, 공연 |
| 15 | Pets & Animals | 반려동물, 동물, 귀여운 영상 |
| 17 | Sports | 스포츠 경기, 하이라이트, 분석 |
| 19 | Travel & Events | 여행, 관광, 이벤트 |
| 20 | Gaming | 게임플레이, 리뷰, e스포츠 |
| 22 | People & Blogs | 일상 브이로그, 개인 채널 |
| 23 | Comedy | 코미디, 개그, 유머 영상 |
| 24 | Entertainment | 예능, 쇼, 엔터테인먼트 |
| 25 | News & Politics | 뉴스, 시사, 정치 |
| 26 | Howto & Style | 요리, 패션, DIY, 메이크업 |
| 27 | Education | 강의, 교육, 학습 콘텐츠 |
| 28 | Science & Technology | 과학, 기술, IT, 실험 |
| 29 | Nonprofits & Activism | 비영리, 사회운동, 캠페인 |

---

### 🔀 예측할 8개 카테고리 (통합)

#### 1️⃣ Music
- **포함된 원본 ID**: `10`  
- **원본 카테고리**: Music  
- **설명**: 음악, 뮤직비디오, 커버, 공연

---

#### 2️⃣ Gaming
- **포함된 원본 ID**: `20`  
- **원본 카테고리**: Gaming  
- **설명**: 게임플레이, 리뷰, e스포츠

---

#### 3️⃣ Entertainment
- **포함된 원본 ID**: `24, 23, 1, 22`  
- **원본 카테고리**:
  - Entertainment (24)
  - Comedy (23)
  - Film & Animation (1)
  - People & Blogs (22)
- **설명**: 예능, 코미디, 영화, 일상 브이로그 등 엔터테인먼트 중심 콘텐츠

---

#### 4️⃣ Education
- **포함된 원본 ID**: `27`  
- **원본 카테고리**: Education  
- **설명**: 강의, 교육, 학습 콘텐츠

---

#### 5️⃣ Science & Technology
- **포함된 원본 ID**: `28`  
- **원본 카테고리**: Science & Technology  
- **설명**: 과학 실험, IT/기술 해설, 정보 전달형 콘텐츠

---

#### 6️⃣ Lifestyle
- **포함된 원본 ID**: `26, 19, 15, 2, 29`  
- **원본 카테고리**:
  - Howto & Style (26)
  - Travel & Events (19)
  - Pets & Animals (15)
  - Autos & Vehicles (2)
  - Nonprofits & Activism (29)
- **설명**: 요리, 패션, DIY, 여행, 반려동물, 자동차 등 실용/생활 콘텐츠

---

#### 7️⃣ Sports
- **포함된 원본 ID**: `17`  
- **원본 카테고리**: Sports  
- **설명**: 스포츠 경기, 하이라이트, 리뷰

---

#### 8️⃣ News & Politics
- **포함된 원본 ID**: `25`  
- **원본 카테고리**: News & Politics  
- **설명**: 뉴스, 시사, 정치 논평


## 1. 라이브러리 설치 및 임포트

In [3]:
# 필요한 패키지 설치
!pip install google-api-python-client pandas python-dotenv tqdm

In [52]:
# 라이브러리 임포트
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import pandas as pd
import json
from datetime import datetime, timezone, timedelta
import time
import re
import logging
from typing import List, Dict, Optional, Union
import os
from tqdm import tqdm
from collections import Counter, defaultdict
from google.colab import userdata

# 로깅 설정 (googleapiclient의 WARNING 숨김)
logging.getLogger('googleapiclient.http').setLevel(logging.ERROR)

# 한국 시간대 정의 for save file
KST = timezone(timedelta(hours=9))

## 2. API 설정

In [5]:
# API Key 설정
API_KEY = userdata.get('YOUTUBE_API_KEY') # colab 사용시, 좌측 열쇠 아이콘 눌러 api 추가 후 사용 가능
# API_KEY = "YOUR_API_KEY_HERE"  # 안되면 여기에 실제 API 키 입력

# YouTube API 클라이언트 생성
youtube = build('youtube', 'v3', developerKey=API_KEY)

print("✅ YouTube API 클라이언트 초기화 완료!")

✅ YouTube API 클라이언트 초기화 완료!


In [53]:
# 간단한 테스트
try:
    request = youtube.videos().list(
        part='snippet',
        chart='mostPopular',
        regionCode='KR',
        maxResults=5
    )
    response = request.execute()

    print(f"✅ API 작동 확인!")
    print(f"인기 급상승 영상 {len(response['items'])}개 조회 성공")
    print(f"\n첫 번째 영상: {response['items'][1]['snippet']['title']}")

except Exception as e:
    print(f"❌ 오류 발생: {e}")
    print("\\n해결 방법:")
    print("1. console.cloud.google.com에서 YouTube Data API v3 활성화")
    print("2. API 키가 올바른지 확인")
    print("3. API 키 제한 설정 확인")

✅ API 작동 확인!
인기 급상승 영상 5개 조회 성공

첫 번째 영상: 죄송한데.. 누구세요..? 마블 ≪원더맨≫ 공식 예고편 리뷰 떡밥 총정리


In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
# 저장 경로 설정
output_dir = '/content/drive/MyDrive/youtube_ml_project'
import os
os.makedirs(output_dir, exist_ok=True)
print(f"✅ 데이터 저장 경로: {output_dir}")

✅ 데이터 저장 경로: /content/drive/MyDrive/youtube_ml_project


## 3. 에러 추적 클래스

In [56]:
class ErrorTracker:
    """수집 과정의 모든 에러를 추적"""

    def __init__(self):
        self.errors = defaultdict(list)
        self.skipped_videos = []
        self.no_comment_videos = []
        self.api_errors = []

    def add_error(self, category: str, error_msg: str, video_id: str = None):
        """에러 기록"""
        self.errors[category].append({
            'error': error_msg,
            'video_id': video_id,
            'timestamp': datetime.now(KST).isoformat()
        })

    def add_no_comment(self, video_id: str, title: str):
        """댓글 없는 영상 기록"""
        self.no_comment_videos.append({'video_id': video_id, 'title': title})

    def add_skip(self, video_id: str, reason: str):
        """스킵된 영상 기록"""
        self.skipped_videos.append({'video_id': video_id, 'reason': reason})

    def get_summary(self) -> str:
        """에러 요약 생성"""
        summary = []
        summary.append("\n" + "="*60)
        summary.append("🔍 에러 및 스킵 통계")
        summary.append("="*60)

        if self.no_comment_videos:
            summary.append(f"\n📝 댓글 비활성화 영상: {len(self.no_comment_videos)}개")

        if self.skipped_videos:
            summary.append(f"⏭️  스킵된 영상: {len(self.skipped_videos)}개")
            for skip in self.skipped_videos[:5]:  # 최대 5개만
                summary.append(f"   - {skip['video_id']}: {skip['reason']}")

        if self.errors:
            summary.append(f"\n❌ 에러 발생 카테고리: {len(self.errors)}개")
            for category, error_list in self.errors.items():
                summary.append(f"   - {category}: {len(error_list)}건")

        if not self.errors and not self.skipped_videos:
            summary.append("\n✅ 에러 없이 모든 데이터 수집 완료!")

        summary.append("="*60)
        return "\n".join(summary)

# 전역 에러 트래커
error_tracker = ErrorTracker()

## 4. 기본 유틸리티 함수

In [57]:
def get_category_name(category_id: str, region_code: str = 'KR') -> str:
    """
    카테고리 ID를 이름으로 변환

    YouTube 카테고리 ID는 전세계 공통이므로 hardcoded 매핑 사용
    (API 호출 불필요 + regionCode 충돌 방지)
    """
    category_map = {
        '1': 'Film & Animation',
        '2': 'Autos & Vehicles',
        '10': 'Music',
        '15': 'Pets & Animals',
        '17': 'Sports',
        '18': 'Short Movies',
        '19': 'Travel & Events',
        '20': 'Gaming',
        '21': 'Videoblogging',
        '22': 'People & Blogs',
        '23': 'Comedy',
        '24': 'Entertainment',
        '25': 'News & Politics',
        '26': 'Howto & Style',
        '27': 'Education',
        '28': 'Science & Technology',
        '29': 'Nonprofits & Activism',
        '30': 'Movies',
        '31': 'Anime/Animation',
        '32': 'Action/Adventure',
        '33': 'Classics',
        '34': 'Documentary',
        '35': 'Drama',
        '36': 'Family',
        '37': 'Foreign',
        '38': 'Horror',
        '39': 'Sci-Fi/Fantasy',
        '40': 'Thriller',
        '41': 'Shorts',
        '42': 'Shows',
        '43': 'Trailers'
    }

    return category_map.get(category_id, f"Unknown (ID: {category_id})")

## 5. 인기 급상승 영상 수집

In [58]:
def get_trending_videos(region_code: str = 'KR',
                       max_results: int = 50,
                       category_id: Optional[str] = None,
                       silent: bool = False) -> List[str]:
    """
    인기 급상승 영상의 Video ID 수집 (에러 처리 강화)
    """
    video_ids = []

    try:
        params = {
            'part': 'id,snippet',
            'chart': 'mostPopular',
            'regionCode': region_code,
            'maxResults': min(50, max_results)
        }

        if category_id:
            params['videoCategoryId'] = category_id

        request = youtube.videos().list(**params)

        while request and len(video_ids) < max_results:
            response = request.execute()

            for item in response['items']:
                video_ids.append(item['id'])

                if len(video_ids) >= max_results:
                    break

            if 'nextPageToken' in response and len(video_ids) < max_results:
                params['pageToken'] = response['nextPageToken']
                params['maxResults'] = min(50, max_results - len(video_ids))
                request = youtube.videos().list(**params)
            else:
                break

        if not silent:
            print(f"✅ 인기 급상승 영상 {len(video_ids)}개 수집 완료 (지역: {region_code})")
        return video_ids

    except HttpError as e:
        error_msg = f"Trending 수집 실패 (지역: {region_code}): {str(e)}"
        error_tracker.add_error('trending', error_msg)
        if not silent:
            print(f"❌ {error_msg}")
        return []

    except Exception as e:
        error_msg = f"예상치 못한 오류: {str(e)}"
        error_tracker.add_error('trending', error_msg)
        if not silent:
            print(f"❌ {error_msg}")
        return []

## 6. 카테고리별 검색

In [59]:
def search_videos_by_category(category_id: str,
                              max_results: int = 50,
                              order: str = 'viewCount',
                              region_code: str = 'KR',
                              published_after: Optional[str] = None,
                              silent: bool = False) -> List[str]:
    """
    특정 카테고리의 인기 영상 수집 (mostPopular 방식)

    v3.1 개선:
    - search.list → videos.list (chart='mostPopular') 변경
    - videoCategoryId가 제대로 작동!
    - API 비용 100 units → 1 unit (99% 절감)
    """
    video_ids = []

    try:
        # mostPopular 방식으로 카테고리별 인기 영상 수집
        params = {
            'part': 'id,snippet',
            'chart': 'mostPopular',
            'regionCode': region_code,
            'videoCategoryId': category_id,
            'maxResults': min(50, max_results)
        }

        request = youtube.videos().list(**params)

        while request and len(video_ids) < max_results:
            response = request.execute()

            if not response.get('items'):
                if not silent:
                    category_name = get_category_name(category_id, region_code)
                    print(f"⚠️  '{category_name}' 카테고리에서 영상을 찾을 수 없음 (지역: {region_code})")
                break

            for item in response['items']:
                video_ids.append(item['id'])

                if len(video_ids) >= max_results:
                    break

            # 페이지네이션
            if 'nextPageToken' in response and len(video_ids) < max_results:
                params['pageToken'] = response['nextPageToken']
                params['maxResults'] = min(50, max_results - len(video_ids))
                request = youtube.videos().list(**params)
            else:
                break

        category_name = get_category_name(category_id, region_code)
        if not silent:
            print(f"✅ '{category_name}' 카테고리 영상 {len(video_ids)}개 수집 완료 (지역: {region_code})")
        return video_ids

    except HttpError as e:
        error_msg = f"검색 실패 (카테고리: {category_id}, 지역: {region_code}): {str(e)}"
        error_tracker.add_error('category_search', error_msg, category_id)
        if not silent:
            print(f"❌ {error_msg}")
        return []

    except Exception as e:
        error_msg = f"예상치 못한 오류: {str(e)}"
        error_tracker.add_error('category_search', error_msg, category_id)
        if not silent:
            print(f"❌ {error_msg}")
        return []

## 7. 채널 영상 수집

In [60]:
def get_channel_videos(channel_id: str,
                      max_results: int = 50,
                      order: str = 'date',
                      silent: bool = False) -> List[str]:
    """
    특정 채널의 영상 수집 (에러 처리 강화)
    """
    video_ids = []

    try:
        params = {
            'part': 'id',
            'channelId': channel_id,
            'type': 'video',
            'order': order,
            'maxResults': min(50, max_results)
        }

        request = youtube.search().list(**params)

        while request and len(video_ids) < max_results:
            response = request.execute()

            for item in response['items']:
                if item['id']['kind'] == 'youtube#video':
                    video_ids.append(item['id']['videoId'])

                if len(video_ids) >= max_results:
                    break

            if 'nextPageToken' in response and len(video_ids) < max_results:
                params['pageToken'] = response['nextPageToken']
                params['maxResults'] = min(50, max_results - len(video_ids))
                request = youtube.search().list(**params)
            else:
                break

        if not silent:
            print(f"✅ 채널 '{channel_id}'의 영상 {len(video_ids)}개 수집 완료")
        return video_ids

    except HttpError as e:
        error_msg = f"채널 수집 실패: {str(e)}"
        error_tracker.add_error('channel', error_msg, channel_id)
        if not silent:
            print(f"❌ {error_msg}")
        return []

    except Exception as e:
        error_msg = f"예상치 못한 오류: {str(e)}"
        error_tracker.add_error('channel', error_msg, channel_id)
        if not silent:
            print(f"❌ {error_msg}")
        return []

## 8. 영상 상세 정보 수집

In [61]:
def get_video_details(video_id: str) -> Optional[Dict]:
    """YouTube 영상의 상세 정보 수집 (에러 처리 강화)"""
    try:
        request = youtube.videos().list(
            part='snippet,statistics,contentDetails',
            id=video_id
        )
        response = request.execute()

        if not response['items']:
            error_tracker.add_error('video_not_found', f"영상을 찾을 수 없음", video_id)
            return None

        video = response['items'][0]
        snippet = video['snippet']
        statistics = video['statistics']
        content_details = video['contentDetails']

        video_data = {
            'video_id': video_id,
            'title': snippet['title'],
            'description': snippet['description'],
            'category_id': snippet['categoryId'],
            'tags': snippet.get('tags', []),
            'published_at': snippet['publishedAt'],
            'channel_id': snippet['channelId'],
            'channel_title': snippet['channelTitle'],
            'view_count': int(statistics.get('viewCount', 0)),
            'like_count': int(statistics.get('likeCount', 0)),
            'comment_count': int(statistics.get('commentCount', 0)),
            'duration': content_details['duration'],
            'collected_at': datetime.now(KST).isoformat()
        }

        return video_data

    except HttpError as e:
        error_tracker.add_error('video_details', str(e), video_id)
        return None

    except Exception as e:
        error_tracker.add_error('video_details', str(e), video_id)
        return None


def get_video_comments(video_id: str, max_comments: int = 100) -> tuple[List[Dict], bool]:
    """
    YouTube 영상의 댓글 수집

    Returns:
        (댓글 리스트, 댓글_활성화_여부)
    """
    comments = []
    comments_enabled = True

    try:
        request = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=min(100, max_comments),
            order='relevance',
            textFormat='plainText'
        )

        while request and len(comments) < max_comments:
            response = request.execute()

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']

                comment_data = {
                    'video_id': video_id,
                    'comment_id': item['id'],
                    'author': comment['authorDisplayName'],
                    'text': comment['textDisplay'],
                    'like_count': comment['likeCount'],
                    'published_at': comment['publishedAt'],
                    'reply_count': item['snippet']['totalReplyCount']
                }

                comments.append(comment_data)

                if len(comments) >= max_comments:
                    break

            if 'nextPageToken' in response and len(comments) < max_comments:
                request = youtube.commentThreads().list(
                    part='snippet',
                    videoId=video_id,
                    maxResults=min(100, max_comments - len(comments)),
                    pageToken=response['nextPageToken'],
                    order='relevance',
                    textFormat='plainText'
                )
            else:
                break

        return comments, comments_enabled

    except HttpError as e:
        if 'commentsDisabled' in str(e):
            comments_enabled = False
            return [], comments_enabled
        else:
            error_tracker.add_error('comments', str(e), video_id)
            return [], True  # 에러지만 비활성화는 아님

    except Exception as e:
        error_tracker.add_error('comments', str(e), video_id)
        return [], True

## 9. 🚀 통합 자동 수집 함수 (v3 - 에러 처리 강화)

In [62]:
def auto_collect_dataset(config: Dict) -> List[Dict]:
    """
    설정에 따라 자동으로 데이터셋 수집 (v3 - 에러 처리 강화)

    새로운 옵션:
    - include_no_comments: 댓글 없는 영상 포함 여부 (기본값: True)
    - min_comment_count: 최소 댓글 수 (기본값: 0)
    - silent_mode: 경고 메시지 숨김 (기본값: False)

    config 예시:
    {
        'trending': {'count': 200, 'region': 'US'},
        'categories': {
            '10': {'count': 100, 'region': 'KR'},
            '20': 100,
        },
        'max_comments_per_video': 100,
        'include_no_comments': True,  # 댓글 없어도 포함
        'min_comment_count': 0,  # 최소 댓글 수
        'silent_mode': False,  # 경고 메시지 출력
        'delay': 1,
        'region': 'KR'
    }
    """
    global error_tracker
    error_tracker = ErrorTracker()  # 초기화

    all_video_ids = set()
    all_data = []

    # 설정 추출
    global_region = config.get('region', 'KR')
    include_no_comments = config.get('include_no_comments', True)
    min_comment_count = config.get('min_comment_count', 0)
    silent = config.get('silent_mode', False)

    print("="*60)
    print("🤖 자동 데이터 수집 시작 (v3)")
    print(f"전역 Region: {global_region}")
    print(f"댓글 없는 영상 포함: {'예' if include_no_comments else '아니오'}")
    if min_comment_count > 0:
        print(f"최소 댓글 수: {min_comment_count}개")
    print("="*60)

    # 1. 인기 급상승 영상 수집
    if 'trending' in config:
        print("\n[1단계] 인기 급상승 영상 수집 중...")
        trending_config = config['trending']
        trending_region = trending_config.get('region', global_region)
        trending_count = trending_config.get('count', 50) if isinstance(trending_config, dict) else trending_config

        print(f"  → Trending Region: {trending_region}")
        video_ids = get_trending_videos(
            region_code=trending_region,
            max_results=trending_count,
            silent=silent
        )
        all_video_ids.update(video_ids)
        time.sleep(config.get('delay', 1))

    # 2. 카테고리별 검색
    if 'categories' in config:
        print(f"\n[2단계] 카테고리별 영상 검색 중...")
        print(f"  → 기본 Region: {global_region}")

        for category_id, category_config in config['categories'].items():
            if isinstance(category_config, dict):
                count = category_config.get('count', 50)
                category_region = category_config.get('region', global_region)
            else:
                count = category_config
                category_region = global_region

            print(f"  → Category {category_id}: Region={category_region}")
            video_ids = search_videos_by_category(
                category_id=category_id,
                max_results=count,
                region_code=category_region,
                silent=silent
            )
            all_video_ids.update(video_ids)
            time.sleep(config.get('delay', 1))

    # 3. 채널별 수집
    if 'channels' in config:
        print("\n[3단계] 특정 채널 영상 수집 중...")
        for channel_id, count in config['channels'].items():
            video_ids = get_channel_videos(
                channel_id=channel_id,
                max_results=count,
                silent=silent
            )
            all_video_ids.update(video_ids)
            time.sleep(config.get('delay', 1))

    print(f"\n{'='*60}")
    print(f"✅ 총 {len(all_video_ids)}개의 고유한 영상 ID 수집 완료")
    print(f"{'='*60}")

    # 4. 각 영상의 상세 정보 + 댓글 수집
    print("\n[4단계] 영상 상세 정보 및 댓글 수집 중...")
    print("(진행 중에는 출력이 일시 중단됩니다)\n")

    max_comments = config.get('max_comments_per_video', 100)

    # tqdm 사용 (깔끔한 진행률)
    for video_id in tqdm(list(all_video_ids), desc="영상 처리", ncols=80):
        try:
            # 영상 정보
            video_info = get_video_details(video_id)
            if not video_info:
                error_tracker.add_skip(video_id, "영상 정보 조회 실패")
                continue

            # 카테고리 이름
            video_info['category_name'] = get_category_name(video_info['category_id'])

            # 댓글
            comments, comments_enabled = get_video_comments(video_id, max_comments)

            # 댓글 비활성화 영상 처리
            if not comments_enabled:
                error_tracker.add_no_comment(video_id, video_info['title'])
                if not include_no_comments:
                    error_tracker.add_skip(video_id, "댓글 비활성화")
                    continue

            # 최소 댓글 수 확인
            if len(comments) < min_comment_count:
                error_tracker.add_skip(video_id, f"댓글 수 부족 ({len(comments)}개)")
                continue

            # 데이터 통합
            data = {
                'video_info': video_info,
                'comments': comments,
                'collection_summary': {
                    'total_comments_collected': len(comments),
                    'comments_enabled': comments_enabled,
                    'collection_date': datetime.now(KST).isoformat()
                }
            }

            all_data.append(data)
            time.sleep(config.get('delay', 1))

        except Exception as e:
            error_tracker.add_error('processing', str(e), video_id)
            continue

    # 진행률 끝
    print(f"\n{'='*60}")
    print(f"✅ 최종 수집 완료: {len(all_data)}개 영상")
    if len(all_video_ids) - len(all_data) > 0:
        print(f"⏭️  스킵됨: {len(all_video_ids) - len(all_data)}개 영상")
    print(f"{'='*60}")

    # 에러 통계 출력
    print(error_tracker.get_summary())

    return all_data

## 10. 데이터 저장 (에러 리포트 포함)

In [63]:
def save_dataset(all_data: List[Dict], output_dir: str = 'youtube_dataset'):
    """
    수집한 전체 데이터셋 저장 (에러 리포트 포함)
    """
    os.makedirs(output_dir, exist_ok=True)

    timestamp = datetime.now(KST).strftime('%Y%m%d_%H%M%S')

    print(f"\n{'='*60}")
    print("💾 데이터 저장 중...")
    print(f"{'='*60}")

    # 1. 전체 데이터 JSON 저장
    json_file = f"{output_dir}/full_dataset_{timestamp}.json"
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)
    print(f"✅ 전체 데이터: {json_file}")

    # 2. 영상 정보만 CSV
    videos = [d['video_info'] for d in all_data]
    videos_df = pd.DataFrame(videos)
    videos_csv = f"{output_dir}/videos_{timestamp}.csv"
    videos_df.to_csv(videos_csv, index=False, encoding='utf-8-sig')
    print(f"✅ 영상 정보 CSV: {videos_csv}")

    # 3. 모든 댓글 통합 CSV
    all_comments = []
    for d in all_data:
        all_comments.extend(d['comments'])

    comments_df = None
    if all_comments:
        comments_df = pd.DataFrame(all_comments)
        comments_csv = f"{output_dir}/comments_{timestamp}.csv"
        comments_df.to_csv(comments_csv, index=False, encoding='utf-8-sig')
        print(f"✅ 댓글 CSV: {comments_csv}")
    else:
        print(f"⚠️  댓글 데이터 없음 (CSV 생성 안함)")

    # 4. 통계 리포트
    report_file = generate_collection_report(all_data, output_dir, timestamp)

    # 5. 에러 리포트 저장
    if error_tracker.errors or error_tracker.skipped_videos:
        error_file = f"{output_dir}/error_report_{timestamp}.json"
        error_data = {
            'errors': dict(error_tracker.errors),
            'skipped_videos': error_tracker.skipped_videos,
            'no_comment_videos': error_tracker.no_comment_videos
        }
        with open(error_file, 'w', encoding='utf-8') as f:
            json.dump(error_data, f, ensure_ascii=False, indent=2)
        print(f"📋 에러 리포트: {error_file}")

    print(f"{'='*60}\n")

    return videos_df, comments_df


def generate_collection_report(all_data: List[Dict], output_dir: str, timestamp: str) -> str:
    """
    수집 통계 리포트 생성
    """
    report = []
    report.append("="*60)
    report.append("YouTube 데이터 수집 리포트")
    report.append("="*60)
    report.append(f"수집 일시: {datetime.now(KST).strftime('%Y-%m-%d %H:%M:%S')}")
    report.append(f"총 영상 수: {len(all_data)}개")

    # 카테고리별 분포
    categories = [d['video_info']['category_name'] for d in all_data]
    category_counts = Counter(categories)
    report.append("\n[카테고리별 영상 수]")
    for cat, count in category_counts.most_common():
        report.append(f"  - {cat}: {count}개")

    # 댓글 통계
    total_comments = sum(len(d['comments']) for d in all_data)
    videos_with_comments = sum(1 for d in all_data if len(d['comments']) > 0)
    avg_comments = total_comments / len(all_data) if all_data else 0

    report.append(f"\n[댓글 통계]")
    report.append(f"  - 총 댓글 수: {total_comments:,}개")
    report.append(f"  - 댓글 있는 영상: {videos_with_comments}개")
    report.append(f"  - 영상당 평균: {avg_comments:.1f}개")

    # 조회수 통계
    view_counts = [d['video_info']['view_count'] for d in all_data]
    report.append(f"\n[조회수 통계]")
    report.append(f"  - 평균: {sum(view_counts)/len(view_counts):,.0f}")
    report.append(f"  - 최대: {max(view_counts):,}")
    report.append(f"  - 최소: {min(view_counts):,}")

    report.append("="*60)

    # 리포트 저장
    report_text = "\n".join(report)
    report_file = f"{output_dir}/report_{timestamp}.txt"
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write(report_text)

    print(f"\n{report_text}")
    print(f"\n✅ 리포트: {report_file}")

    return report_file

## 11. 실행 예제

### 예제 1: 댓글 필터링 포함 (추천)

In [ ]:
# 댓글 없는 영상 제외
config_filtered = {
    'trending': {'count': 10, 'region': 'KR'},
    'categories': {
        '10': {'count': 10, 'region': 'KR'},
        '20': {'count': 10, 'region': 'KR'},
    },
    'max_comments_per_video': 20,
    'include_no_comments': False,  # 댓글 없으면 제외!
    'min_comment_count': 5,  # 최소 5개 댓글 필요
    'delay': 1,
    'region': 'KR'
}

# 실행
dataset = auto_collect_dataset(config_filtered)
save_dataset(dataset, 'youtube_filtered')

🤖 자동 데이터 수집 시작 (v3)
전역 Region: KR
댓글 없는 영상 포함: 아니오
최소 댓글 수: 5개

[1단계] 인기 급상승 영상 수집 중...
  → Trending Region: KR
✅ 인기 급상승 영상 10개 수집 완료 (지역: KR)

[2단계] 카테고리별 영상 검색 중...
  → 기본 Region: KR
  → Category 10: Region=KR
✅ 'Music' 카테고리 영상 10개 수집 완료 (지역: KR)
  → Category 20: Region=KR
✅ 'Gaming' 카테고리 영상 10개 수집 완료 (지역: KR)

✅ 총 25개의 고유한 영상 ID 수집 완료

[4단계] 영상 상세 정보 및 댓글 수집 중...
(진행 중에는 출력이 일시 중단됩니다)



영상 처리: 100%|████████████████████████████████| 25/25 [00:22<00:00,  1.13it/s]


✅ 최종 수집 완료: 18개 영상
⏭️  스킵됨: 7개 영상

🔍 에러 및 스킵 통계

📝 댓글 비활성화 영상: 7개
⏭️  스킵된 영상: 7개
   - 6tXCKF33IzM: 댓글 비활성화
   - aTqchg0BUG4: 댓글 비활성화
   - TLftyCObb_E: 댓글 비활성화
   - -QdWntrZ4KY: 댓글 비활성화
   - 4WFsxzp7SBc: 댓글 비활성화

💾 데이터 저장 중...
✅ 전체 데이터: youtube_filtered/full_dataset_20251013_213003.json
✅ 영상 정보 CSV: youtube_filtered/videos_20251013_213003.csv
✅ 댓글 CSV: youtube_filtered/comments_20251013_213003.csv

YouTube 데이터 수집 리포트
수집 일시: 2025-10-13 21:30:03
총 영상 수: 18개

[카테고리별 영상 수]
  - Gaming: 9개
  - Music: 5개
  - Film & Animation: 2개
  - Entertainment: 2개

[댓글 통계]
  - 총 댓글 수: 360개
  - 댓글 있는 영상: 18개
  - 영상당 평균: 20.0개

[조회수 통계]
  - 평균: 3,888,396
  - 최대: 37,060,690
  - 최소: 21,805

✅ 리포트: youtube_filtered/report_20251013_213003.txt
📋 에러 리포트: youtube_filtered/error_report_20251013_213003.json



(       video_id                                              title  \
 0   EmeW6li6bbo                    NMIXX(엔믹스) “Blue Valentine” M/V   
 1   xvRpIebeul4        엥 드래곤은 사기 아닌가요? : 드래곤타입 하트골드【포켓몬 모든타입 깨기】#1   
 2   ux3QETpLcPs  劇場版『チェンソーマン レゼ篇』オープニングムービー 　主題歌：米津玄師「IRIS OUT」...   
 3   qw_JJRWLnII              운명을 바꿀 최후의 격량🌊 | [탁류] 8-9회 예고편 | 디즈니+   
 4   qptvuPRIBuI  한국최초 "음양 스푸키 펌키"가 떴습니다!!! 모든 할로윈 시크릿 얻고 새로나온 여...   
 5   4AFjS2EyfHw                                   이 독재자의 끝을 아는 사람:   
 6   KULUkjwXqBs                       본캐 레알 급여 290에 맞춰서 새롭게 단장 합니다   
 7   zM8fkvomoWI  아무도 예상못한 WER 프리미어 "김된모 비토", 상대도 당황한 슈퍼 크랙 플레이ㅋ...   
 8   1hIR75GsjFA                          마인크래프트, 코마가 서버의 신이 되는 PVP   
 9   viwIK-mzO9E              스틸 볼 런: 죠죠의 기묘한 모험 | 공식 티저 예고편 | 넷플릭스   
 10  9VGivoFZE-c         신을 재조립하라.. 영화처럼 보는 SCP 이야기 [SCP-001/부서진 신]   
 11  ylTuLv9iDmY                             임영웅 -  우리에게 안녕   
 12  2cvvmywR0HI  "유튜브가 인정한 합법 야동" 쏟아지는 저격들을 피하고 이상호는 미션을 성공할 수 ...   
 13  U

### 예제 2: 모든 영상 포함 (기본)

In [30]:
# 댓글 없어도 포함
config_all = {
    'trending': {'count': 10, 'region': 'KR'},
    'categories': {
        '10': {'count': 10, 'region': 'KR'},
    },
    'max_comments_per_video': 20,
    'include_no_comments': True,  # 댓글 없어도 포함 (기본값)
    'delay': 1,
    'region': 'KR'
}

# 실행
# dataset = auto_collect_dataset(config_all)
# save_dataset(dataset, 'youtube_all')

### 예제 3: 조용한 모드 (경고 숨김)

In [31]:
# 경고 메시지 최소화
config_silent = {
    'trending': {'count': 10, 'region': 'KR'},
    'categories': {
        '10': 10,
    },
    'max_comments_per_video': 20,
    'silent_mode': True,  # 경고 숨김
    'delay': 1,
    'region': 'KR'
}

# 실행
# dataset = auto_collect_dataset(config_silent)
# save_dataset(dataset, 'youtube_silent')

### 예제 0: 다국어 혼합 데이터셋 (가장 유연함)

In [32]:
# 각 카테고리별로 다른 지역 설정
config_multinational = {
    'trending': {
        'count': 100,
        'region': 'US'  # 미국 트렌드
    },
    'categories': {
        '10': {'count': 100, 'region': 'KR'},  # 한국 음악
        '20': {'count': 100, 'region': 'US'},  # 미국 게임
        '24': {'count': 100, 'region': 'JP'},  # 일본 엔터테인먼트
        '25': {'count': 100, 'region': 'KR'},  # 한국 뉴스
        '28': {'count': 100, 'region': 'US'},  # 미국 과학기술
    },
    'max_comments_per_video': 100,
    'delay': 1,
    'region': 'KR'  # 기본값 (명시하지 않은 것들에 적용)
}

# 실행 (주석 해제하여 사용)
# dataset = auto_collect_dataset(config_multinational)
# save_dataset(dataset, 'youtube_multinational')

### 예제 02: 한국 중심 (전역 설정)

In [33]:
# 간단한 방식: 모든 카테고리에 전역 region 적용
config_korea = {
    'trending': {'count': 200},  # region 없음 → 전역 KR 사용
    'categories': {
        '10': 100,  # Music
        '20': 100,  # Gaming
        '22': 100,  # People & Blogs
        '24': 100,  # Entertainment
        '25': 100,  # News & Politics
        '26': 100,  # Howto & Style
        '28': 100,  # Science & Technology
    },
    'region': 'KR',  # 전역: 모두 한국
    'max_comments_per_video': 100,
    'delay': 1
}

# 실행
# dataset = auto_collect_dataset(config_korea)
# save_dataset(dataset, 'youtube_korea')

### 예제 test: 빠른 테스트용

In [68]:
# 소규모 테스트
config_test = {
    'trending': {'count': 10, 'region': 'KR'},
    'categories': {
        '10': {'count': 10, 'region': 'KR'},
        '20': {'count': 10, 'region': 'US'},
    },
    'max_comments_per_video': 20,
    'include_no_comments': False,  # 댓글 없으면 제외!
    'min_comment_count': 5,  # 최소 5개 댓글 필요
    'delay': 1,
    'region': 'KR'
}

# 실행
test_dataset = auto_collect_dataset(config_test)
save_dataset(test_dataset, 'youtube_test')

🤖 자동 데이터 수집 시작 (v3)
전역 Region: KR
댓글 없는 영상 포함: 아니오
최소 댓글 수: 5개

[1단계] 인기 급상승 영상 수집 중...
  → Trending Region: KR
✅ 인기 급상승 영상 10개 수집 완료 (지역: KR)

[2단계] 카테고리별 영상 검색 중...
  → 기본 Region: KR
  → Category 10: Region=KR
✅ 'Music' 카테고리 영상 10개 수집 완료 (지역: KR)
  → Category 20: Region=US
✅ 'Gaming' 카테고리 영상 10개 수집 완료 (지역: US)

✅ 총 28개의 고유한 영상 ID 수집 완료

[4단계] 영상 상세 정보 및 댓글 수집 중...
(진행 중에는 출력이 일시 중단됩니다)



영상 처리: 100%|████████████████████████████████| 28/28 [00:24<00:00,  1.15it/s]


✅ 최종 수집 완료: 20개 영상
⏭️  스킵됨: 8개 영상

🔍 에러 및 스킵 통계

📝 댓글 비활성화 영상: 7개
⏭️  스킵된 영상: 8개
   - 6tXCKF33IzM: 댓글 비활성화
   - aTqchg0BUG4: 댓글 비활성화
   - TLftyCObb_E: 댓글 비활성화
   - koBgYLlnVFc: 댓글 수 부족 (3개)
   - -QdWntrZ4KY: 댓글 비활성화

💾 데이터 저장 중...
✅ 전체 데이터: youtube_test/full_dataset_20251013_215443.json
✅ 영상 정보 CSV: youtube_test/videos_20251013_215443.csv
✅ 댓글 CSV: youtube_test/comments_20251013_215443.csv

YouTube 데이터 수집 리포트
수집 일시: 2025-10-13 21:54:43
총 영상 수: 20개

[카테고리별 영상 수]
  - Gaming: 12개
  - Music: 5개
  - Film & Animation: 2개
  - Entertainment: 1개

[댓글 통계]
  - 총 댓글 수: 400개
  - 댓글 있는 영상: 20개
  - 영상당 평균: 20.0개

[조회수 통계]
  - 평균: 3,818,443
  - 최대: 37,238,438
  - 최소: 22,668

✅ 리포트: youtube_test/report_20251013_215443.txt
📋 에러 리포트: youtube_test/error_report_20251013_215443.json



(       video_id                                              title  \
 0   EmeW6li6bbo                    NMIXX(엔믹스) “Blue Valentine” M/V   
 1   RGbDW-hyxxc           How well do I know ROBLOX | ALBERTGUESSR   
 2   ux3QETpLcPs  劇場版『チェンソーマン レゼ篇』オープニングムービー 　主題歌：米津玄師「IRIS OUT」...   
 3   qw_JJRWLnII              운명을 바꿀 최후의 격량🌊 | [탁류] 8-9회 예고편 | 디즈니+   
 4   ZRMpPp6U-NY                        My Grandma Tried To Kill Me   
 5   4AFjS2EyfHw                                   이 독재자의 끝을 아는 사람:   
 6   wTYDBFO0y78                             Silent Hill f - Part 3   
 7   Ig_fJ0eJe_8              Giving YouTubers Their Own BRAINROTS!   
 8   1hIR75GsjFA                          마인크래프트, 코마가 서버의 신이 되는 PVP   
 9   nboz6eLKoaQ  Avatar Legends: The Fighting Game - Official A...   
 10  fbGeCfHG648                                       I AM PIBBLE.   
 11  viwIK-mzO9E              스틸 볼 런: 죠죠의 기묘한 모험 | 공식 티저 예고편 | 넷플릭스   
 12  ylTuLv9iDmY                             임영웅 -  우리에게 안녕   
 13  U

### 예제 work: 바로 사용 가능한 코드 (800개 가량 빠른 수집)


In [65]:
config_fast_run = {
    'trending': {'count': 200, 'region': 'KR'},
    'categories': {
        # Music (1개 → Music)
        '10': {'count': 150, 'region': 'KR'},

        # Gaming (1개 → Gaming)
        '20': {'count': 150, 'region': 'KR'},

        # Entertainment (4개 → Entertainment)
        '24': {'count': 100, 'region': 'KR'},  # Entertainment
        '23': {'count': 50, 'region': 'KR'},   # Comedy
        '1': {'count': 30, 'region': 'KR'},    # Film
        '22': {'count': 50, 'region': 'KR'},   # People & Blogs

        # Education (1개 → Education)
        '27': {'count': 80, 'region': 'KR'},

        # Science & Technology (1개 → Science & Technology)
        '28': {'count': 80, 'region': 'KR'},

        # Lifestyle (5개 → Lifestyle)
        '26': {'count': 50, 'region': 'KR'},   # Howto & Style
        '15': {'count': 50, 'region': 'KR'},   # Pets
        '19': {'count': 30, 'region': 'KR'},   # Travel
        '2': {'count': 20, 'region': 'KR'},    # Autos
        '29': {'count': 20, 'region': 'KR'},   # Nonprofit

        # Sports (1개 → Sports)
        '17': {'count': 100, 'region': 'KR'},

        # News & Politics (1개 → News & Politics)
        '25': {'count': 50, 'region': 'KR'},
    },
    'max_comments_per_video': 100,
    'include_no_comments': False,
    'min_comment_count': 10,
    'delay': 1,
    'region': 'KR'
}

# 실행
# dataset = auto_collect_dataset(config_fast_run)
# save_dataset(dataset, 'youtube_fast_run')

## 13. API 할당량 계산기

In [35]:
def calculate_quota_usage(config: Dict) -> Dict:
    """
    설정 기반 API 할당량 사용량 계산 (v3 - 필터링 고려)

    v3 개선사항:
    - include_no_comments와 min_comment_count 고려
    - 실제 수집될 영상 수 예측
    - 더 정확한 할당량 계산
    """
    usage = {
        'trending': 0,
        'category_search': 0,
        'channel_search': 0,
        'video_details': 0,
        'comments': 0,
        'category_lookup': 0,
        'total': 0
    }

    # 1. 인기 급상승
    if 'trending' in config:
        trending_config = config['trending']
        count = trending_config.get('count', 0) if isinstance(trending_config, dict) else trending_config
        # Trending은 50개당 1 unit (매우 저렴!)
        usage['trending'] = ((count - 1) // 50 + 1) * 1

    # 2. 카테고리 검색
    if 'categories' in config:
        for category_config in config['categories'].values():
            count = category_config.get('count', 0) if isinstance(category_config, dict) else category_config
            # Search는 50개당 100 units (비쌈!)
            # v3.1: videos.list(mostPopular) = 1 unit per request
            usage['category_search'] += ((count - 1) // 50 + 1) * 1

    # 3. 채널 검색
    if 'channels' in config:
        for ch_count in config['channels'].values():
            usage['channel_search'] += ((ch_count - 1) // 50 + 1) * 100

    # 4. 총 영상 수 추정
    total_videos = 0
    if 'trending' in config:
        trending_config = config['trending']
        count = trending_config.get('count', 0) if isinstance(trending_config, dict) else trending_config
        total_videos += count

    if 'categories' in config:
        for category_config in config['categories'].values():
            count = category_config.get('count', 0) if isinstance(category_config, dict) else category_config
            total_videos += count

    if 'channels' in config:
        total_videos += sum(config['channels'].values())

    # 5. 필터링 고려 (예측)
    include_no_comments = config.get('include_no_comments', True)
    min_comment_count = config.get('min_comment_count', 0)

    # 댓글 비활성화율: 약 20-30% (경험적 수치)
    no_comment_rate = 0.25
    # 최소 댓글 수 미달률: min_comment_count에 따라 다름
    insufficient_comment_rate = min(0.3, min_comment_count * 0.02) if min_comment_count > 0 else 0

    if not include_no_comments:
        # 댓글 없는 영상 제외
        expected_videos = int(total_videos * (1 - no_comment_rate))
    else:
        expected_videos = total_videos

    if min_comment_count > 0:
        # 최소 댓글 수 미달 제외
        expected_videos = int(expected_videos * (1 - insufficient_comment_rate))

    # 6. 영상 상세 정보 (중복 제거 고려)
    # 실제로는 trending과 categories에서 중복이 약 10-20% 발생
    unique_videos = int(total_videos * 0.9)  # 중복 10% 가정
    usage['video_details'] = unique_videos * 1

    # 7. 카테고리 조회 (고유 카테고리 수만큼)
    unique_categories = len(config.get('categories', {}))
    usage['category_lookup'] = unique_categories * 1

    # 8. 댓글 수집
    max_comments = config.get('max_comments_per_video', 100)
    # 100개당 1 unit
    comment_units_per_video = ((max_comments - 1) // 100 + 1)
    usage['comments'] = unique_videos * comment_units_per_video

    # 총합
    usage['total'] = sum(v for k, v in usage.items() if k != 'total')

    # 출력
    print("="*60)
    print("📊 API 할당량 사용 예상 (v3)")
    print("="*60)
    print(f"\n[수집 계획]")
    print(f"  목표 영상 수: {total_videos}개")
    print(f"  중복 제거 후: 약 {unique_videos}개 (중복 10% 가정)")

    if not include_no_comments or min_comment_count > 0:
        print(f"\n[필터링 예상]")
        if not include_no_comments:
            print(f"  댓글 비활성화 제외: 약 {int(no_comment_rate*100)}% → {int(unique_videos * no_comment_rate)}개 제외")
        if min_comment_count > 0:
            print(f"  최소 {min_comment_count}개 댓글 미달 제외: 약 {int(insufficient_comment_rate*100)}% → {int(unique_videos * insufficient_comment_rate)}개 제외")
        print(f"  → 최종 예상 수집: 약 {expected_videos}개")

    print(f"\n[API 사용량 상세]")
    print(f"  1. Trending 조회: {usage['trending']} units")
    print(f"  2. Category 검색: {usage['category_search']} units ⚠️")
    print(f"  3. Channel 검색: {usage['channel_search']} units")
    print(f"  4. 영상 상세 정보: {usage['video_details']} units")
    print(f"  5. 카테고리 조회: {usage['category_lookup']} units")
    print(f"  6. 댓글 수집: {usage['comments']} units")

    print(f"\n[총계]")
    print(f"  총 예상 사용량: {usage['total']:,} units")
    print(f"  일일 할당량 대비: {(usage['total']/10000)*100:.1f}%")

    if usage['total'] > 10000:
        days_needed = (usage['total'] - 1) // 10000 + 1
        print(f"\n  ⚠️  {days_needed}일에 걸쳐 수집 필요")
        print(f"  💡 일일 약 {total_videos // days_needed}개 영상씩 수집 권장")
    else:
        remaining = 10000 - usage['total']
        print(f"\n  ✅ 하루에 충분히 가능!")
        print(f"  💡 남은 할당량: {remaining:,} units ({(remaining/10000)*100:.1f}%)")

    # 비용 절감 팁
    if usage['category_search'] > 1000:
        print(f"\n[💰 비용 절감 팁]")
        print(f"  ⚠️  Category 검색이 {usage['category_search']} units 사용 중")
        print(f"  💡 v3.1에서 99% 절감됨 (v3: {usage['category_search']*100} units)")
        print(f"  대안 1: Trending 비중 늘리기 (1 unit!)")
        print(f"  대안 2: 카테고리 수 줄이기")
        print(f"  대안 3: 여러 날 분산 수집")

    print("="*60)

    return usage

In [36]:
usage = calculate_quota_usage(config_test)

📊 API 할당량 사용 예상 (v3)

[수집 계획]
  목표 영상 수: 30개
  중복 제거 후: 약 27개 (중복 10% 가정)

[필터링 예상]
  댓글 비활성화 제외: 약 25% → 6개 제외
  최소 5개 댓글 미달 제외: 약 10% → 2개 제외
  → 최종 예상 수집: 약 19개

[API 사용량 상세]
  1. Trending 조회: 1 units
  2. Category 검색: 2 units ⚠️
  3. Channel 검색: 0 units
  4. 영상 상세 정보: 27 units
  5. 카테고리 조회: 2 units
  6. 댓글 수집: 27 units

[총계]
  총 예상 사용량: 59 units
  일일 할당량 대비: 0.6%

  ✅ 하루에 충분히 가능!
  💡 남은 할당량: 9,941 units (99.4%)


In [66]:
usage = calculate_quota_usage(config_fast_run)

📊 API 할당량 사용 예상 (v3)

[수집 계획]
  목표 영상 수: 1210개
  중복 제거 후: 약 1089개 (중복 10% 가정)

[필터링 예상]
  댓글 비활성화 제외: 약 25% → 272개 제외
  최소 10개 댓글 미달 제외: 약 20% → 217개 제외
  → 최종 예상 수집: 약 725개

[API 사용량 상세]
  1. Trending 조회: 4 units
  2. Category 검색: 23 units ⚠️
  3. Channel 검색: 0 units
  4. 영상 상세 정보: 1089 units
  5. 카테고리 조회: 15 units
  6. 댓글 수집: 1089 units

[총계]
  총 예상 사용량: 2,220 units
  일일 할당량 대비: 22.2%

  ✅ 하루에 충분히 가능!
  💡 남은 할당량: 7,780 units (77.8%)


## 💡 v2 사용 가이드

### Region 우선순위 (중요!)

```
1순위: 카테고리별/Trending별 region
2순위: 전역 region (config['region'])
3순위: 기본값 'KR'
```

### 설정 형식

**카테고리 설정 2가지 방법:**

```python
# 방법 1: 단순 숫자 (count만)
'10': 100  # → 전역 region 사용

# 방법 2: dict (count + region)
'10': {'count': 100, 'region': 'KR'}  # → KR 명시
```

### 실전 활용 시나리오

**시나리오 1: 프로젝트 - 한국 중심**
- 전역 `region='KR'`
- 모든 카테고리에 숫자만 입력
- 간단하고 깔끔

**시나리오 2: 연구 - 문화 비교**
- 같은 카테고리를 여러 지역에서 수집
- 각 카테고리마다 region 명시
- 한국 vs 미국 음악 트렌드 비교 가능

**시나리오 3: 글로벌 데이터셋**
- 미국 트렌드 + 한국 K-POP + 일본 애니메이션
- 카테고리별 region 다르게 설정
- 다양한 문화권 데이터 수집

### 지원 Region 코드

| 코드 | 국가 |
|------|------|
| KR | 대한민국 |
| US | 미국 |
| JP | 일본 |
| GB | 영국 |
| DE | 독일 |
| FR | 프랑스 |
| CA | 캐나다 |
| IN | 인도 |
| BR | 브라질 |

### ⚠️ 주의사항

1. **Region ≠ 언어**: `region='KR'`은 "한국에서 인기" (영어 영상 포함 가능)
2. **API 비용**: 카테고리 검색은 100 units (비쌈!)
3. **중복 제거**: 자동으로 처리됨 (set 사용)
4. **할당량 관리**: `calculate_quota_usage()` 먼저 실행 권장

## 💡 v3 개선사항 요약

### 1. 출력 개선
- ✅ 진행률 표시와 메시지 분리
- ✅ googleapiclient의 WARNING 숨김
- ✅ 깔끔한 구조화된 출력

### 2. 에러 처리
- ✅ 모든 API 호출에 try-except
- ✅ 카테고리 조회 실패 시 fallback 매핑
- ✅ 검색 결과 없음 처리
- ✅ 영상 찾을 수 없음 처리

### 3. 댓글 필터링
- ✅ `include_no_comments`: 댓글 없는 영상 제외 옵션
- ✅ `min_comment_count`: 최소 댓글 수 설정
- ✅ 댓글 비활성화 영상 추적

### 4. 에러 리포트
- ✅ 모든 에러 추적 및 기록
- ✅ 스킵된 영상 목록
- ✅ 에러 통계 출력
- ✅ JSON 형태로 저장

### 발견 가능한 에러들

#### API 관련
- `quotaExceeded`: 할당량 초과
- `commentsDisabled`: 댓글 비활성화
- `videoNotFound`: 영상 찾을 수 없음

#### 데이터 관련
- 카테고리 조회 실패
- 검색 결과 없음
- 영상 정보 조회 실패

#### 필터 관련
- 댓글 수 부족
- 댓글 비활성화

### 설정 예시 비교

```python
# 프로젝트용 (높은 품질)
config = {
    'trending': {'count': 200, 'region': 'KR'},
    'categories': {...},
    'include_no_comments': False,  # 댓글 필수
    'min_comment_count': 10,  # 최소 10개
    'max_comments_per_video': 100
}

# 연구용 (모든 데이터)
config = {
    'trending': {'count': 200, 'region': 'KR'},
    'categories': {...},
    'include_no_comments': True,  # 모두 포함
    'min_comment_count': 0,  # 제한 없음
    'max_comments_per_video': 100
}
```

# Google Cloud Console API 설정 가이드 (Colab용)

## 🎯 필요한 API

### ✅ 필수 API (1개)
1. **YouTube Data API v3** - 영상 정보, 댓글, 검색 등 모든 기능

### ⚠️ 선택 API (필요시)
- 없음 (YouTube Data API v3만 있으면 충분!)

---

## 📋 단계별 설정 가이드

### Step 1: Google Cloud Console 접속
1. https://console.cloud.google.com 접속
2. Google 계정으로 로그인

### Step 2: 프로젝트 생성
1. 상단 프로젝트 드롭다운 클릭
2. "새 프로젝트" 클릭
3. 프로젝트 이름 입력 (예: `youtube-ml-project`)
4. "만들기" 클릭
5. 생성된 프로젝트 선택

### Step 3: YouTube Data API v3 활성화
1. 좌측 메뉴에서 "API 및 서비스" > "라이브러리" 클릭
2. 검색창에 `YouTube Data API v3` 입력
3. "YouTube Data API v3" 클릭
4. **"사용" 또는 "사용 설정"** 버튼 클릭
5. 활성화 완료! (몇 초 소요)

### Step 4: API 키 생성
1. 좌측 메뉴에서 "API 및 서비스" > "사용자 인증 정보" 클릭
2. 상단 "+ 사용자 인증 정보 만들기" 클릭
3. **"API 키"** 선택
4. API 키가 생성됨 → **복사하여 안전하게 보관!**

### Step 5: API 키 제한 설정 (권장)
생성된 API 키 옆의 편집(연필) 아이콘 클릭

#### 5-1. API 제한 설정
- "API 제한사항" 섹션에서
- "키 제한" 선택
- "YouTube Data API v3" 체크
- "저장" 클릭

#### 5-2. 애플리케이션 제한사항 (선택)
- 보안을 위해 IP 주소 제한 가능
- Colab은 IP가 자주 바뀌므로 **"없음"** 권장

---

## 🔑 API 키 사용법 (Colab)

### 방법 1: 직접 입력 (테스트용)
```python
API_KEY = "YOUR_API_KEY_HERE"  # 여기에 발급받은 키 붙여넣기
```

### 방법 2: Colab Secrets 사용 (권장) ⭐
```python
from google.colab import userdata
API_KEY = userdata.get('YOUTUBE_API_KEY')
```

**Colab Secrets 설정 방법:**
1. Colab 노트북에서 좌측 🔑 아이콘 클릭
2. "+ 새 보안 비밀 추가" 클릭
3. 이름: `YOUTUBE_API_KEY`
4. 값: 발급받은 API 키 붙여넣기
5. "노트북 액세스 허용" 토글 ON

### 방법 3: 환경변수 (로컬용)
```python
import os
API_KEY = os.getenv('YOUTUBE_API_KEY')
```

---

## 📊 API 할당량 정보

### 기본 할당량
- **일일 할당량: 10,000 units**
- 매일 오전 12:00 (PST, 태평양 표준시)에 리셋
- 한국 시간으로는 오후 5시 (여름) / 오후 6시 (겨울)

### API 비용 (Units)

| 작업 | 비용 |
|------|------|
| `videos().list` (영상 정보) | 1 unit |
| `videos().list` (chart=mostPopular) | 1 unit |
| `search().list` (검색) | **100 units** |
| `commentThreads().list` (댓글) | 1 unit |
| `videoCategories().list` (카테고리) | 1 unit |


### 할당량 초과 시
- `HttpError 403: quotaExceeded` 에러 발생
- 다음 날까지 대기 또는
- 추가 API 키 발급 (다른 프로젝트)

---

## 🚨 주의사항 및 팁

### 보안
1. **API 키 노출 금지**
   - GitHub에 업로드 금지
   - 공개 Colab 노트북에 직접 입력 금지
   - Secrets 기능 사용 권장

2. **API 키 제한 설정**
   - YouTube Data API v3만 사용하도록 제한
   - 불필요한 권한 차단

3. **키 유출 시**
   - 즉시 Cloud Console에서 키 삭제
   - 새로운 키 발급

### 할당량 관리
1. **비용이 비싼 작업 최소화**
   - `search().list`는 100 units (비쌈!)
   - `videos().list`는 1 unit (저렴)
   - Trending 활용 권장

2. **진행 상황 저장**
   - 중간에 중단돼도 이어서 가능
   - JSON 파일로 주기적 저장

3. **여러 날 분산**
   - 1,000개 목표 → 2-3일에 걸쳐 수집
   - 매일 300-400개씩

### Colab 환경 주의사항
1. **세션 타임아웃**
   - 90분 유휴 시 세션 종료
   - 12시간 최대 실행 시간
   - 중간 저장 필수!

2. **런타임 재시작 시**
   - API 키 재설정 필요
   - Secrets 사용하면 자동

3. **데이터 저장**
   - Colab 세션 종료 시 데이터 삭제
   - Google Drive 연동 권장

---

## 📦 Colab에서 시작하기

### 완전한 초기 설정 코드
```python
# 1. 패키지 설치
!pip install google-api-python-client pandas tqdm -q

# 2. API 키 설정 (Secrets 사용)
from google.colab import userdata
API_KEY = userdata.get('YOUTUBE_API_KEY')

# 3. YouTube API 클라이언트 초기화
from googleapiclient.discovery import build
youtube = build('youtube', 'v3', developerKey=API_KEY)

print("✅ 설정 완료!")

# 4. 테스트
try:
    request = youtube.videos().list(
        part='snippet',
        chart='mostPopular',
        regionCode='KR',
        maxResults=5
    )
    response = request.execute()
    print(f"✅ API 작동 확인: {len(response['items'])}개 영상 조회 성공!")
except Exception as e:
    print(f"❌ 오류: {e}")
```

### Google Drive 연동 (데이터 저장용)
```python
from google.colab import drive
drive.mount('/content/drive')

# 데이터 저장 경로
output_dir = '/content/drive/MyDrive/youtube_dataset'
```

---

## 🔍 문제 해결

### "API가 활성화되지 않았습니다" 오류
→ Step 3 다시 확인: YouTube Data API v3 활성화

### "잘못된 API 키" 오류
→ API 키 복사 시 공백 포함 여부 확인

### "할당량 초과" 오류
→ Cloud Console > API 및 서비스 > 대시보드에서 사용량 확인

### Colab에서 "userdata.get() 실패"
→ Secrets 설정 후 "노트북 액세스 허용" 토글 확인

---

## 📚 추가 자료

- [YouTube Data API 공식 문서](https://developers.google.com/youtube/v3)
- [API 할당량 정책](https://developers.google.com/youtube/v3/determine_quota_cost)
- [Google Cloud Console](https://console.cloud.google.com)

---

## ✅ 체크리스트

설정 완료 확인:
- [ ] Google Cloud Console 프로젝트 생성
- [ ] YouTube Data API v3 활성화
- [ ] API 키 발급
- [ ] API 키 제한 설정 (YouTube Data API v3만)
- [ ] Colab Secrets에 API 키 저장
- [ ] 테스트 코드 실행 성공
- [ ] Google Drive 연동 (선택)

모든 체크 완료 → 데이터 수집 시작! 🚀